In [1]:
import os
import yaml
import logging
import numpy as np
import importlib
import subprocess
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [8]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.utils import ml_utils, eda_utils

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = ml_utils.load_config(config_path)

## 3.1. Загрузка данных

In [9]:
# Загрузка train и test
df_train = ml_utils.data_load(data_type='train', config=config)
df_test = ml_utils.data_load(data_type='test', config=config)

In [10]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Id,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,230,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28
1,231,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59
2,232,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82
3,233,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76
4,234,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40


In [11]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Id,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28
1,1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7
2,2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100
3,3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365
4,4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28


In [12]:
# Удаление неинформативного признака Id
df_train = df_train.drop(columns=["Id"])
df_test = df_test.drop(columns=["Id"])

In [13]:
# Удаление дубликатов
df_train_cleaned = df_train.drop_duplicates()
test_cleaned = df_test.drop_duplicates()

## 3.2. Создание инженерных признаков

In [14]:
# Создание признаков (w_c, SP_pct)
df_train_feat = eda_utils.add_concrete_ratios(df_train_cleaned)
df_test_feat = eda_utils.add_concrete_ratios(df_test)

df_train_feat.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


## 3.3. Оценка объема данных и выбор алгоритмов

In [15]:
# Формируем список алгоритмов исходя из объема данных по правилу NEPV
models = ml_utils.check_models_by_nepv(df_train_feat, config)

[REGRESSION] Правило NEPV: 781 наблюдений / 11 признаков = 71.00
LinearRegression / Ridge / Lasso — соответствуют (≥ 20)
CHAID (не реализован в sklearn) — соответствует (≥ 50)
Сложные модели (RF, Boosting и др.) — не соответствуют (< 200)
DecisionTreeRegressor (CART) — добавлен с осторожностью. NEPV к нему не применяется строго.

Список рекомендованных моделей: ['LinearRegression', 'Ridge', 'Lasso', 'DecisionTreeRegressor']


## 3.4. Проверка порядка признаков в тренировочном и тестовом датасете

In [16]:
# Создание массива из признаков и массива из целевой переменной
X = df_train_feat.drop(columns=["Strength"])
y = df_train_feat["Strength"]

In [17]:
# Сравниваем порядок признаков в тренировочном и тестовом датасете
if list(X.columns) == list(df_test_feat.columns):
    print("Порядок признаков совпадает")
else:
    print("Порядок признаков отличается")

Порядок признаков совпадает


## 3.5. Генерация EDA отчета

#### Предварительный отчет

In [ ]:
# Общая информация
df_train_feat.info()

In [ ]:
# Проверка на пропуски train
df_train_feat.isna().sum()

In [ ]:
# Проверка на пропуски test
df_train_feat.isna().sum()

In [ ]:
# Основные статистики train
df_train_feat.describe()

#### Генерация подробного отчета ydataprofaling

In [ ]:
# Генерация отчета по тренировочным данным с помощью ydata-profiling
ml_utils.eda_report(df_train_feat, "train", config)

## 3.6. Анализ выбросов

#### Анализ выбросов

In [ ]:
# Построение сводной таблицы по выбросам на оснвове IQR и значений ГОСТ
summary_df,  outlier_masks_df = eda_utils.detect_outliers(df_train_feat, config)
summary_df

In [ ]:
# Построение графиков 
ml_utils.plot_outliers(df_train_feat, summary_df, max_plots=10)

In [ ]:
# Сохранение отчетов
output_dir=Path().resolve().parent / config["output"]["eda_report_dir"]

ml_utils.save_outliers_report(summary_df, output_dir=output_dir)

## 3.7. Анализ нулей в признаках

#### Анализ признаков с нулнвыми значениями

In [ ]:
config = eda_utils.analyze_zeros(df_train_feat, config)

## 3.8. Анализ зависимости целевой переменнной от признаков

### Анализ корреляции целевой переменнной и признаков

In [ ]:
# Создает комплексный датафрейм c результатами анализа
# корреляции признаков и целевой переменной
df_corr_target = ml_utils.create_feature_analysis(X, y)
df_corr_target

**Выводы:** Признак Fly Ash показывает очень слабую и статистически незначимую связь с целевой переменной Strength. Поэтому его стоит объединить с другими признаками или удалить

In [ ]:
# Визуализация анализа признаков
ml_utils.visualize_feature_analysis(df_corr_target) 

In [ ]:
# Визуализируем тренды зависимостей таргета от признаков
df_trend_results = ml_utils.plot_feature_trends(df_train_feat,
                                                config, 
                                                target='Strength',
                                                figsize=(16, 50),
                                                alpha=0.25)

In [ ]:
# Выводим датасет преобразований призгнаков и значений R2
df_trend_results

In [ ]:
# Добавляем новые значения в существующий config
new_transformations = dict(zip(df_trend_results['feature'],
                               df_trend_results['best_transformation']))

config['best_transformations'].update(new_transformations)

### Анализ мультиколлинеарности признаков

### Объявление функции

In [ ]:
def check_multicollinearity(df: pd.DataFrame,
                            target_column: pd.Series, 
                            threshold: int = 10,
                            plot_heatmap: bool = True) -> pd.DataFrame:
    """
    Проверяет мультиколлинеарность для списка признаков в DataFrame.

    Parameters:
    df: Исходный DataFrame с данными.
    target_column: Целевая переменная.
    threshold: Пороговое значение VIF для выделения проблемных признаков. По умолчанию 10.
    plot_heatmap: Строить ли тепловую карту корреляций. По умолчанию True.

    Returns:
    pd.DataFrame: DataFrame с признаками и их VIF значениями, отсортированный по убыванию VIF.
    """

    features = df.select_dtypes(include='number').columns.tolist()
    target_data = target_column
    
    # Шаг 1: Матрица корреляций и тепловая карта
    if plot_heatmap:
        # Вычисляем матрицу корреляций только для нужных признаков
        corr_matrix = df[features].corr()
        plt.figure(figsize=(10, 8))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Маска для верхнего треугольника
        sns.heatmap(corr_matrix,
                    mask=mask,
                    annot=True,
                    fmt='.2f',
                    cmap='coolwarm',
                    center=0,
                    square=True)
        plt.title('Матрица корреляций (верхний треугольник)')
        plt.tight_layout()
        plt.show()

    # Шаг 2: Расчет VIF
    # statsmodels требует добавления константы для расчета VIF
    X = add_constant(df[features])
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X.columns
    
    # Рассчитываем VIF для каждого признака
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    
    # Убираем константу из результатов (ее VIF будет огромным, но она не нужна)
    vif_data = vif_data[vif_data['Feature'] != 'const']
    vif_data = vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)

    return vif_data

In [ ]:
def highlight_high_vif(vif_data: pd.DataFrame, 
                       high_threshold: int = 10, 
                       medium_threshold: int = 5) -> pd.io.formats.style.Styler:
    """
    Визуально выделяет признаки с высоким VIF в таблице с градацией цветов.
    
    Parameters:
        vif_data (pd.DataFrame): DataFrame с колонками ['Feature', 'VIF']
        high_threshold (int): Порог для красного выделения (VIF >= 10). По умолчанию 10
        medium_threshold (int): Порог для желтого выделения (5 <= VIF < 10). По умолчанию 5
    
    Returns:
        pd.io.formats.style.Styler: Стилизованный DataFrame для отображения
    """
    
    def _highlight_vif_gradient(s):
        """Внутренняя функция для применения градиентного стиля"""
        styles = []
        for vif_value in s:
            if vif_value >= high_threshold:
                styles.append('background-color: #ffcccc')
            elif medium_threshold <= vif_value < high_threshold:
                styles.append('background-color: #fff2cc')
            else:
                styles.append('')
        return styles
    
    styled_vif = (vif_data.style
                  .apply(_highlight_vif_gradient, subset=['VIF'])
                  .format({'VIF': '{:.2f}'}))
    
    return styled_vif

### Анализ мультиколлинеарности

In [18]:
# вывод матрицы корреляций
vif_results = check_multicollinearity(df=X,
                                      target_column=y,
                                      threshold=5)

NameError: name 'check_multicollinearity' is not defined

In [ ]:
# Отображаем значения VIF для признаков
styled_vif = highlight_high_vif(vif_results)
display(styled_vif)

**Выводы:** высокая корреляция между признаками Superplasticizer и Sp/C_pct; а также W/C и Cement. Основной вариант действий: удаление признаков Cement и Superplasticizer